In [1]:
import pandas as pd

In [188]:
stop_times = pd.read_csv('/Users/alex/Downloads/LittleRoady_GTSF/stop_times.txt')

In [21]:
stops = pd.read_csv('/Users/alex/Downloads/LittleRoady_GTSF/stops.txt')

In [59]:
stop_names = stops.set_index('stop_id').stop_name
aug_stop_times = stop_times.assign(stop_name=stop_times.stop_id.map(lambda v: stop_names[v]))

In [63]:
import geopandas as gpd
import shapely as shp

In [84]:
aug_stops = gpd.GeoDataFrame(
    stops,
    geometry=[shp.geometry.Point(x, y) for (_, (y, x)) in stops[['stop_lat', 'stop_lon']].iterrows()]
)

In [88]:
aug_stops

,stop_id,stop_name,stop_lat,stop_lon,geometry
0,LittleRoady-201905060001,RIDOT,41.831750,-71.414200,POINT (-71.41419999999999 41.83175)
1,LittleRoady-201905060002,Promenade,41.828650,-71.420314,POINT (-71.420314 41.82865)
2,LittleRoady-201905060003,ALCO,41.829120,-71.431341,POINT (-71.4313405 41.82912)
3,LittleRoady-201905060004,Eagle Square,41.825125,-71.439420,POINT (-71.43942 41.825125)
4,LittleRoady-201905060005,Rising Sun Mills,41.821700,-71.440100,POINT (-71.4401 41.8217)
5,LittleRoady-201905060006,Olneyville Square,41.816800,-71.442917,POINT (-71.44291700000001 41.8168)
6,LittleRoady-201905060007,US Rubber,41.826450,-71.438275,POINT (-71.438275 41.82645)
7,LittleRoady-201905060008,Capco,41.828460,-71.428143,POINT (-71.42814300000001 41.82846)
8,LittleRoady-201905060009,Providence Place,41.828475,-71.421990,POINT (-71.42199000000001 41.828475)
9,LittleRoady-201905060010,Providence Station,41.829516,-71.413585,POINT (-71.413585 41.829516)


In [104]:
pd.set_option('max_rows', 500)

In [283]:
# list(aug_stop_times.groupby('trip_id'))[0][1]

In [284]:
# list(aug_stop_times.groupby('trip_id'))[1][1]

In [285]:
# len(list(aug_stop_times.groupby('trip_id')))

In [286]:
# list(aug_stop_times.groupby('trip_id'))[2][1]

In [287]:
# list(aug_stop_times.groupby('trip_id'))[3][1]

In [288]:
# list(aug_stop_times.groupby('trip_id'))[4][1]

In [289]:
# list(aug_stop_times.groupby('trip_id'))[5][1]

In [290]:
# list(aug_stop_times.groupby('trip_id'))[6][1]

In [291]:
# list(aug_stop_times.groupby('trip_id'))[7][1]

In [119]:
pd.read_csv('/Users/alex/Downloads/LittleRoady_GTSF/routes.txt')

,route_id,agency_id,route_type,route_short_name,route_long_name
0,LittleRoady-2019-2020,LittleRoady-20190513,3,LR,LittleRoady


In [130]:
RIDOT_idx = stop_times.stop_id.str[-2:] == '01'
Olneyville_Square_idx = stop_times.stop_id.str[-2:] == '06'

In [138]:
import numpy as np
np.where(RIDOT_idx)[0][1:]

array([  5,  17,  29,  41,  53,  65,  77,  89, 101, 113, 125, 137, 149,
       161, 173, 181, 193, 205, 217, 225, 237, 249, 261, 273, 291, 303,
       315, 327, 339, 351, 363, 375, 387, 399, 411, 423, 435, 447, 459,
       471, 483, 495, 507, 519, 531, 543, 555, 567, 579, 591, 603, 615,
       631, 643, 655, 667, 679, 691, 703, 715, 727, 739, 751, 763, 775,
       787, 803, 815, 827, 839, 851, 863, 869, 881, 893, 905, 917, 929])

In [139]:
np.where(Olneyville_Square_idx)[0]

array([ 10,  22,  34,  46,  58,  70,  82,  94, 106, 118, 130, 142, 154,
       166, 186, 198, 210, 222, 230, 242, 254, 266, 278, 284, 296, 308,
       320, 332, 344, 356, 368, 380, 392, 404, 416, 428, 440, 452, 464,
       476, 488, 500, 512, 524, 536, 548, 560, 572, 584, 596, 608, 620,
       624, 636, 648, 660, 672, 684, 696, 708, 720, 732, 744, 756, 768,
       780, 792, 808, 820, 832, 844, 856, 874, 886, 898, 910, 922, 934])

In [199]:
n = ['06', '05', '07', '03', '08', '09', '10', '01']
n_transitions = []
for i in range(1, len(n)):
    n_transitions.append([n[i - 1], n[i]])
    
s = ['01', '02', '03', '04', '05', '06']
s_transitions = []
for i in range(1, len(s)):
    s_transitions.append([s[i - 1], s[i]])


def get_edges(inp):
    edges = []

    prior_stop_id = None
    prior_edge = None
    for i in range(len(inp)):
        stop_id = inp.iloc[i].stop_id

        if prior_stop_id == None:
            prior_stop_id = stop_id
            prior_i = i
            continue
        elif stop_id[-2:] == '01':
            edges.append((prior_i, i + 1, 'N'))
            prior_stop_id = stop_id
            prior_i = i
        elif stop_id[-2:] == '06':
            edges.append((prior_i, i + 1, 'S'))
            prior_stop_id = stop_id
            prior_i = i

    if prior_i != i:
        if ([inp.iloc[i - 1].stop_id[-2:], inp.iloc[i].stop_id[-2:]] in s_transitions):
            edges.append((prior_i, i, 'S'))
        elif ([inp.iloc[i - 1].stop_id[-2:], inp.iloc[i].stop_id[-2:]] in n_transitions):
            edges.append((prior_i, i, 'N'))
        else:
            raise ValueError
        
    return edges

In [200]:
edgelists = []
for i in range(8):
    edgelists.append(get_edges(list(aug_stop_times.groupby('trip_id'))[i][1]))

In [259]:
stop_times_sliced_by_trip_id = [tup[1] for tup in list(aug_stop_times.groupby('trip_id'))]

In [300]:
new_stop_times = []

for (slice_idx, edgelist) in enumerate(edgelists):
    for (trip_idx, (first, last, direction)) in enumerate(edgelist):
        orig_record = stop_times_sliced_by_trip_id[slice_idx].iloc[first:last]
        if len(orig_record) > 1:
            new_stop_times.append(
                orig_record.assign(
                    trip_id=orig_record.trip_id.map(lambda v: v + f'_{trip_idx}_{direction}'),
                    stop_sequence=[v + 1 for v in range(len(orig_record))]
                )
            )

new_stop_times = pd.concat(new_stop_times).drop('stop_name', axis='columns')

In [311]:
trip_ids = new_stop_times.trip_id.unique()

new_trips = pd.DataFrame({
    'route_id': ['LittleRoady-2019-2020'] * len(trip_ids),
    'service_id': [1] * len(trip_ids),
    'trip_id': trip_ids,
})

In [306]:
%mkdir data/

In [313]:
new_trips.to_csv('data/trips.txt', index=False)

In [314]:
len(stop_times)

936

In [315]:
len(new_stop_times)

1080

In [317]:
new_stop_times.to_csv('data/stop_times.txt', index=False)

...then copy over the original files I got sent for all of the other info (routes, agency, calendar, etcetera).